# **Red Neuronal Convolucional Dentem Tutor**


> *Creado por Lex Ortega*



Clasificador binario que permite determinar entre dos posibilidades la mas probable

# Erosion VS comopleto
clase1 = erosion

clase2 =  completo

-----------------------------------------------------

### Como ejecutar:

1. Ir ejecutando el procedimiento para que prepare todo, cuando se indique, en la carpeta **originales** de deben de cargar las imagenes que corresponden con las categorías que se esta trabajando (Especificadas previamente).


Para un reproceso, se ejecuta el bloque del codigo que vacía las carpetas, cabe mencionar que cualquiera se puede vaciar, excepto la carpeta de **originales**


In [ ]:
#Importar librerias
import cv2
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.preprocessing.image import image

# para manejo de archivos
import glob
from google.colab import files
import os
from os import path

In [ ]:
if path.exists('/content/originales/clase1') == False:
  os.makedirs('/content/originales/clase1')

if path.exists('/content/originales/clase2') == False:
  os.makedirs('/content/originales/clase2')

if path.exists('/content/cla1VScla2/clase1') == False:
  os.makedirs('/content/cla1VScla2/clase1')

if path.exists('/content/cla1VScla2/clase2') == False:
  os.makedirs('/content/cla1VScla2/clase2')       

if path.exists('/content/aumento/clase1') == False:
  os.makedirs('/content/aumento/clase1')  

if path.exists('/content/aumento/clase2') == False:
  os.makedirs('/content/aumento/clase2')    

**%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%**

#                 Cargar imágenes en las carpetas

**%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%**

In [ ]:
# Indicar las etiquetas para cada clase
etiq_clase1 = "erosion"
etiq_clase2 = "completo"

# Determinar rutas donde se encuentran las imagenes y donde van a quedar
origen_clase1= "/content/originales/clase1"
origen_clase2= "/content/originales/clase2"

filtro_clase1= "/content/cla1VScla2/clase1"
filtro_clase2= "/content/cla1VScla2/clase2"

data_aumentation_clase1 = "/content/aumento/clase1"
data_aumentation_clase2 = "/content/aumento/clase2"


# Listas de imagenes para procesar
listaClase1=[]
listaClase2=[]

# Preparar el arreglo de imágenes 
imagenes = []
# Determinar el tamaño que deben de tener las imagenes
tamanio = 100

In [ ]:
#Vaciar carpetas para reprocesar
from os import remove

for name in glob.glob(filtro_clase1 + '/*'):
  remove(name)

for name in glob.glob(filtro_clase2 + '/*'):
    remove(name)     

for name in glob.glob(data_aumentation_clase1 + '/*'):
  remove(name)

for name in glob.glob(data_aumentation_clase2 + '/*'):
    remove(name) 

In [ ]:
# Encontrar y listar imagenes originales
orig_listaClase1=[]

for name in glob.glob(origen_clase1 + '/*'):
    orig_listaClase1.append(name)


orig_listaClase2=[]
for name in glob.glob(origen_clase2 + '/*'):
    orig_listaClase2.append(name)


#print(orig_listaClase1)
print ('Número de archivos originales en clase1: ', len(orig_listaClase1)) 

#print(orig_listaClase2)
print ('Número de archivos originales en clase2: ', len(orig_listaClase2)) 

Número de archivos originales en clase1:  52
Número de archivos originales en clase2:  327


In [ ]:
#Aumento de datos visibles en carpeta

#data_aumentation_clase1, data_aumentation_clase2  #Carpetas donde se guardarán las imagenes
num_aumento = 6   # Cantidad de imagenes aumentadas por cada imagen encontrada

train_datagen = ImageDataGenerator(
    rotation_range=20,       #Rotacion en grados
    zoom_range=0.2,          # zoom --> 1 = 100%
    width_shift_range=0.1,   # recorrer horizontal
    height_shift_range=0.1,  # recorrer vertical
    horizontal_flip=True,    # giro 90 grados horizontal
    vertical_flip=True)     # giro 90 grados vertical

i=0
num_images=0
for img in orig_listaClase1:

    imge=load_img(img)  #cargar imagenes clase1
    
    imge=cv2.resize(image.img_to_array(imge), (tamanio, tamanio), interpolation = cv2.INTER_AREA)
    x= imge/255
    x=np.expand_dims(x,axis=0)
    t=1
    
    for tratada in train_datagen.flow(x,batch_size=1):
        a=image.img_to_array(tratada[0])
        imagen=tratada[0,:,:]*255
        imgfinal = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
        cv2.imwrite(data_aumentation_clase1+"/%i%i.jpg"%(i,t), imgfinal) 
        t+=1
        
        num_images+=1
        if t>num_aumento:
            break
    i+=1
    
print("Imagenes clase1 generadas: ", num_images)

## -----------------------------------------------------

i=0
num_images=0
for img in orig_listaClase2:

    imge=load_img(img)  #cargar imagenes clase2
    
    imge=cv2.resize(image.img_to_array(imge), (tamanio, tamanio), interpolation = cv2.INTER_AREA)
    x= imge/255
    x=np.expand_dims(x,axis=0)
    t=1
    
    for tratada in train_datagen.flow(x,batch_size=1):
        a=image.img_to_array(tratada[0])
        imagen=tratada[0,:,:]*255
        imgfinal = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
        cv2.imwrite(data_aumentation_clase2+"/%i%i.jpg"%(i,t), imgfinal) 
        t+=1
        
        num_images+=1
        if t>num_aumento:
            break
    i+=1
    
print("Imagenes clase2 generadas: ", num_images)

Imagenes clase1 generadas:  312
Imagenes clase2 generadas:  1962


In [ ]:
# Encontrar y listar imagenes aumentadas
aument_listaClase1=[]

for name in glob.glob(data_aumentation_clase1 + '/*'):
    aument_listaClase1.append(name)


aument_listaClase2=[]
for name in glob.glob(data_aumentation_clase2 + '/*'):
    aument_listaClase2.append(name)


#print(aument_listaClase1)
print ('Número de archivos originales en clase1: ', len(aument_listaClase1)) 

#print(aument_listaClase2)
print ('Número de archivos originales en clase2: ', len(aument_listaClase2)) 

Número de archivos originales en clase1:  312
Número de archivos originales en clase2:  1962


In [ ]:
# Juntar todas las imagenes para ser procesadas
for imgS in orig_listaClase1:
  listaClase1.append(imgS)

for imgE in orig_listaClase2:  
  listaClase2.append(imgE)

for imgS in aument_listaClase1:
  listaClase1.append(imgS)

for imgE in aument_listaClase2:  
  listaClase2.append(imgE)  

In [ ]:
#Cargar imagenes de clase1
cont=0

for img in listaClase1:
  cont=cont+1
  tmp_image = plt.imread(img)[:,:,:3]
  tmp_image = cv2.resize(tmp_image, (tamanio, tamanio))
  imagenes.append(tmp_image)


print ("--------------------------------------------")
print ("Total de imagenes leídas en clase1: ", cont)

--------------------------------------------
Total de imagenes leídas en clase1:  364


In [ ]:
#Cargar imagenes de clase2
cont=0

for img in listaClase2:
  cont=cont+1
  tmp_image = plt.imread(img)[:,:,:3]
  tmp_image = cv2.resize(tmp_image, (tamanio, tamanio))
  imagenes.append(tmp_image)

  #print ("Cargando... " + img)

print ("--------------------------------------------")
print ("Total de imagenes leídas en clase2: ", cont)

--------------------------------------------
Total de imagenes leídas en clase2:  2289


In [ ]:
# Crear etiquetas
etiquetas=[]
indice=[]

for i in listaClase1:
  etiquetas.append(etiq_clase1)
  indice.append(0)
for i in listaClase2:  
  etiquetas.append(etiq_clase2)
  indice.append(1)

print("Cantidad de etiquetas creadas: ", len(etiquetas))
print(etiquetas)
print(indice)

Cantidad de etiquetas creadas:  2653
['erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'erosion', 'eros

In [ ]:
# Convirtiendo etiquetas e imagenes a formato numpy
y = np.array(indice)    # etiquetas
X = np.array(imagenes, dtype=float)    # imagenes

Una vez que se tienen los datos listos y en el formato numpy, se procede a preparar la red neuronal

In [ ]:
# Crear conjuntos de entrenamiento y prueba
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('Shape de entrenamiento : ', train_X.shape, train_Y.shape)
print('Shape de test : ', test_X.shape, test_Y.shape)

Shape de entrenamiento :  (2122, 100, 100, 3) (2122,)
Shape de test :  (531, 100, 100, 3) (531,)


In [ ]:
#Normalizar
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

*******************************************************************************


#**Hasta este punto, se tienen los datos listos para empezar a probar redes.** 


*******************************************************************************

In [ ]:
#Crear los modelos iniciales
#  Sigmoid regresa siempre datos entre 0 y 1 >> si se acerca a 0 es sano, y a 1 es enfermo.

"""modDenso = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(tamanio, tamanio, 3)),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(30, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

modConv = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(tamanio, tamanio, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

modConvDrop = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(tamanio, tamanio, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='selu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])"""

In [ ]:
#Compilar modelos. Usar crossentropy binario ya que tenemos solo 2 opciones (perro o gato)
"""modDenso.compile(optimizer='rmsprop',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modConv.compile(keras.optimizers.RMSprop(learning_rate=1e-5),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modConvDrop.compile(keras.optimizers.RMSprop(learning_rate=1e-5),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])"""

In [ ]:
"""tensorboardDenso = TensorBoard(log_dir='logs/denso')
modDenso.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=50,
                callbacks=[tensorboardDenso])"""

In [ ]:
"""tensorboardConv = TensorBoard(log_dir='logs/conv')
modConv.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=50,
                callbacks=[tensorboardConv])"""

In [ ]:
"""tensorboardConvDrop = TensorBoard(log_dir='logs/convDrop')
modConvDrop.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=50,
                callbacks=[tensorboardConvDrop])"""

In [ ]:
#Cargar la extension de tensorboard de colab
#%load_ext tensorboard

#Ejecutar tensorboard e indicarle que lea la carpeta "logs"
#%tensorboard --logdir logs

# Una vez elegido el tipo de red, se procede a probar diferentes configuraciones


Red Convolucional

In [ ]:
#Crear los modelos iniciales
#  Sigmoid regresa siempre datos entre 0 y 1 >> si se acerca a 0 es sano, y a 1 es enfermo.

modConv1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(tamanio, tamanio, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

modConv2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (2,2), activation='relu', input_shape=(tamanio, tamanio, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (2,2), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (2,2), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (2,2), activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

modConv3 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='selu', input_shape=(tamanio, tamanio, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='selu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='selu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (3,3), activation='selu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='selu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

modConv4 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (2,2), activation='selu', input_shape=(tamanio, tamanio, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (2,2), activation='selu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (2,2), activation='selu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (2,2), activation='selu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='selu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
#Compilar modelos. Usar crossentropy binario ya que tenemos solo 2 opciones 
modConv1.compile(keras.optimizers.RMSprop(learning_rate=1e-5),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modConv2.compile(keras.optimizers.RMSprop(learning_rate=1e-5),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modConv3.compile(keras.optimizers.RMSprop(learning_rate=1e-5),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modConv4.compile(keras.optimizers.RMSprop(learning_rate=1e-5),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [ ]:
tensorConv1 = TensorBoard(log_dir='rmsprop/conv1')
modConv1.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=30,
                callbacks=[tensorConv1])

In [ ]:
tensorConv2 = TensorBoard(log_dir='rmsprop/conv2')
modConv2.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=30,
                callbacks=[tensorConv2])

In [ ]:
tensorConv3 = TensorBoard(log_dir='rmsprop/conv3')
modConv3.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=30,
                callbacks=[tensorConv3])

In [ ]:
tensorConv4 = TensorBoard(log_dir='rmsprop/conv4')
modConv4.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=30,
                callbacks=[tensorConv4])

In [ ]:
#Cargar la extension de tensorboard de colab
%load_ext tensorboard

#Ejecutar tensorboard e indicarle que lea la carpeta "logs"
%tensorboard --logdir rmsprop

Cambiando funcion de activacion

In [ ]:
#Compilar modelos. Usar crossentropy binario ya que tenemos solo 2 opciones 
modConv1.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modConv2.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modConv3.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modConv4.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [ ]:
tensorConv1a = TensorBoard(log_dir='adam/conv1')
modConv1.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=30,
                callbacks=[tensorConv1a])

In [ ]:
tensorConv2a = TensorBoard(log_dir='adam/conv2')
modConv2.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=30,
                callbacks=[tensorConv2a])

In [ ]:
tensorConv3a = TensorBoard(log_dir='adam/conv3')
modConv3.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=30,
                callbacks=[tensorConv3a])

In [ ]:
tensorConv4a = TensorBoard(log_dir='adam/conv4')
modConv4.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=30,
                callbacks=[tensorConv4a])

In [ ]:
#Cargar la extension de tensorboard de colab
%load_ext tensorboard

#Ejecutar tensorboard e indicarle que lea la carpeta "logs"
%tensorboard --logdir adam